### Build Model

#### Define the Q network model

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn.functional as F


class DQN(nn.Module):
    def __init__(self, state_size, hidden_size, action_size):
        super(DQN,self).__init__()
        self.layer1=nn.Linear(state_size,hidden_size)
        self.layer2=nn.Linear(hidden_size,hidden_size)
        self.layer3=nn.Linear(hidden_size,action_size)

    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        return self.layer3(x)

#### Glucose-Insulin Dynamic

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import math

# Define the parameters of the model (example values, you should replace them with actual patient data)
params = {
    "Kxgi": 3.15e-5,  # rate of glucose uptake per insulin concentration
    "Kxi": 0.038,     # disappearance rate constant for insulin
    "Kxg": 0.1,       # rate of insulin-independent glucose uptake
    "VG": 0.18,       # distribution volume for glucose
    "VI": 0.25,       # distribution volume for insulin
    "G_star": 9,      # state when insulin ecretion equal to Tigmax/2
    "gamma": 15.92,   # progressivity pancreas reacts to circulating glucose concentrations
    "tau_g": 6.5,     # Time delay 对照实验中取了0
    "Gb": 8.85,       # mM - Basal glucose
    "Ib": 59.85,       # pM - Basal insulin
    "dt": 1.5
}

# Initial conditions for G and I
G0=params['Gb']
I0=params['Ib']
Y0=[G0,I0]

G_history=[G0]
I_history=[I0]
Y=[G_history[-1],I_history[-1]]

# History buffer for delayed glucose
class GlucoseBuffer:
    def __init__(self, tau_g, initial_glucose, dt):
        self.buffer = np.full(int(tau_g / dt), initial_glucose)
        self.tau_g = tau_g
        self.dt = dt #time step between successive glucose values
        self.pos = 0  # current position in the buffer
    
    def update(self, glucose):
        self.buffer[self.pos] = glucose
        self.pos = (self.pos + 1) % len(self.buffer)
    
    def get_delayed_glucose(self):
        return self.buffer[self.pos]

dt=params['dt']
# Create an instance of the buffer
buffer = GlucoseBuffer(params['tau_g'], initial_glucose=G0,dt=dt)
# The model's equations, considering the time delay
def glucose_insulin_dynamics(t,Y,params,v=0.0):
    # v is the insulin input, you'll need to set it according to the treatment policy
    G=Y[0]
    I=Y[1]
    G0=params['Gb']
    I0=params['Ib']
    # Use buffer to get delayed G(t - tau_g)
    G_delayed = buffer.get_delayed_glucose()

    # Define the φ function
    def phi(G_delayed):
        return (G_delayed / params['G_star'])**params['gamma'] / (1 + (G_delayed / params['G_star'])**params['gamma'])
    
    # define Tgh function
    def Tgh(params,G0,I0):
        return params['VG']*G0*(params['Kxg']+params['Kxgi']*I0)
    
    # define Tigmax function
    def Tigmax(params,I0, G_delayed):
        return (params['VI']*params['Kxi']*I0)/phi(G_delayed)
    
    # Equations for dG/dt and dI/dt
    dG_dt = -params['Kxgi'] * G * I + Tgh(params,G0,I0)/ params['VG']
    dI_dt = -params['Kxi'] * I + (Tigmax(params,I0,G_delayed) * phi(G_delayed)) / params['VI'] + v / params['VI']
    # Update the glucose buffer
    buffer.update(G)

    return [dG_dt, dI_dt]

# Solve the system of equations with solve_ivp, using a custom time stepping to update the buffer
def solve_system(t_span, Y0,v):
    t_eval = np.arange(t_span[0], t_span[1], buffer.dt)
    sol = solve_ivp(glucose_insulin_dynamics, t_span, Y0,args=(params,v,), t_eval=t_eval, method='RK45')
    return sol

# Time span for the simulation (in minutes)
t_span = (0, 15)  # 24 hours

# # Solve the differential equations with the custom solve_system function
solution=solve_system(t_span,Y0,0)
print(solution)

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.500e+00  3.000e+00  4.500e+00  6.000e+00
             7.500e+00  9.000e+00  1.050e+01  1.200e+01  1.350e+01]
        y: [[ 8.850e+00  1.018e+01 ...  1.935e+01  2.065e+01]
            [ 5.985e+01  5.985e+01 ...  5.985e+01  5.985e+01]]
      sol: None
 t_events: None
 y_events: None
     nfev: 20
     njev: 0
      nlu: 0


#### Define reward function

In [3]:
# initial value
N_G=0

In [4]:
def reward(N_G,G):
    if 4<G<7.5:
        return N_G/24
    elif G<2.78 or G>13.9:
        return -1
    else:
        return 0
   
# solution=solve_system(t_span,Y0)
# G=solution.y[0]
# I=solution.y[1]
# G_history=G.tolist()

# print(G_history)
# I_history.append(I)
G_history=[G0]
I_history=[I0]
for G in G_history:
    if 4<G<7.5:
        N_G+=1
print(N_G)

0


#### Optimizer

In [5]:
import torch
learning_rate=0.1
torch_params = {k: torch.tensor(v, dtype=torch.float64, requires_grad=True) for k, v in params.items()}
optimizer = torch.optim.Adam(torch_params.values(), lr=learning_rate)

#### Replay Memory

In [6]:
from collections import namedtuple, deque
import random

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

#### Import Data

Define the object

In [7]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import os

class DiabetesDataset(Dataset):
    def __init__(self, patient_file,transform=None,target_transform=None):
        self.data=pd.read_csv(patient_file)
        self.diabete_record=self.data.iloc[:,1]
        self.labels=self.data.iloc[:,0]
        self.insulin_doses=self.data.iloc[:,2]
        # self.label=torch.arange(0,(len(self.labels)+1)* 1.5, step=1.5)
        self.transform=transform
        self.target_transform=target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):       
        diabete=self.diabete_record.iloc[idx]
        label=self.labels[idx]
        insulin_doses=self.insulin_doses.iloc[idx]
        if self.transform:
            diabete=self.transform(diabete)
        if self.target_transform:
            label=self.target_transform(label)
        return torch.tensor(diabete),insulin_doses



Extract Data

In [8]:
patient1=DiabetesDataset('/home/fyyy0407/RL/Deep_Q_Learning/Shanghai_T2DM/1.csv')

In [9]:
import math
diabete,insulin_doses=patient1.__getitem__(patient1.__len__()-1)
print(insulin_doses)
print(patient1.__len__())

0
1339


#### Training

In [10]:
# import math


# # def select_action(episode):
#     # patient1=DiabetesDataset('/home/fyyy0407/RL/Deep_Q_Learning/Shanghai_T2DM/1.csv')
#     # diabete,label,insulin_doses=patient1.__getitem__(episode)
#     # if math.isnan(insulin_doses):
#     #     insulin_doses=0.0
#     # return diabete, label, insulin_doses

# def select_action(episode):
#     patient1=DiabetesDataset('/home/fyyy0407/RL/Deep_Q_Learning/originData.csv')
#     label,insulin_doses=patient1.__getitem__(episode)
#     return label,insulin_doses
   

# def train_loop(G0,I0,optimizer, episodes, memory, batch_size, gamma, target_dqn=None, update_target_every=10):
#     for episode in range(episodes):
#         # Get initial state from the physiological model
#         current_glucose, current_insulin = G0,I0  # Your function to get initial state
#         total_reward = 0
        
#         # Choose an action based on the current state and policy
#         (time,action) = select_action(episode)  # 利用数据中的action
#         current_Y=[current_glucose,current_insulin]
#         # Apply the action to the environment (administer insulin dose) and observe next state and reward
#         # next_glucose, next_insulin= slove_system(action, state)  # Your function to apply action
#         sol=solve_system(t_span,current_Y)
#         next_G=sol.y[0]
#         next_I=sol.y[1]
#         G_history=next_G.tolist()
#         I_history=next_I.tolist()
#         for G in G_history:
#             if 4<G<7.5:
#                 N_G+=1

#         rwd=reward(N_G,current_glucose)
#         # Store the transition in memory
#         next_Y = torch.tensor([next_G, next_I], dtype=torch.float32)
#         memory.push((current_Y, action, next_Y, rwd))
            
#         # Move to the next state
#         current_Y = next_Y
            
#         # Perform one step of the optimization (sample from memory and update model)
#         optimize_dqn(dqn, optimizer, memory, batch_size, gamma)
            
#         total_reward += rwd
        
#         if target_dqn and episode % update_target_every == 0:
#             # Update the target network with the current policy network
#             target_dqn.load_state_dict(dqn.state_dict())
        
#         # Logging
#         print(f'Episode {episode}: Total reward: {total_reward}')


#     pass

#### Optimizing and Training

In [11]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

parameters_history = []
torch_params = {k: torch.tensor(v, dtype=torch.float32, requires_grad=True) for k, v in params.items()}

train_dataloader=DataLoader(patient1,batch_size=1,shuffle=False)
# loss_fn = nn.CrossEntropyLoss()
loss_fn = torch.nn.MSELoss()
def train_loop(G_history,I_history,batch_size,train_dataloader,solve_system,loss_fn,optimizer):
    size=train_dataloader.__len__()
    for batch, (glc,isl)in enumerate(train_dataloader):
        crt_G=G_history[-1]
        crt_I=I_history[-1]
        crt_Y=[crt_G,crt_I]
        print(batch)

        pred=solve_system(t_span,crt_Y,v=isl.item())
        nxt_G=pred.y[0]
        nxt_I=pred.y[1]
        G_history.append(nxt_G.tolist())
        I_history.append(nxt_I.tolist())

        crt_G_tensor = torch.tensor([crt_G], dtype=torch.float32,requires_grad=True)  # Ensure it's a float tensor
        glc_tensor = torch.tensor([glc], dtype=torch.float32)  # Ensure it's a float tensor
        loss = loss_fn(crt_G_tensor, glc_tensor)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Record the current parameters
        current_params = {k: v.detach().clone() for k, v in torch_params.items()}
        parameters_history.append(current_params)

        if batch%100==0:
            loss,current=loss.item(),batch*batch_size+len(glc)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
train_loop(G_history,I_history,1,train_dataloader,solve_system,loss_fn,optimizer)

0
loss: 16835.062500  [    1/ 1339]
1


ValueError: `y0` must be 1-dimensional.